In [2]:
library(shiny)
library(visNetwork)
library(leaflet)
library(ggplot2)
library(maps)
library(dplyr)
library(tidyr)
library(dygraphs)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [3]:
contrib <- read.csv('odp_contributionsbygender.csv',check.names=FALSE)

head(contrib)

Contribution_ID,ISOCode3,M49_Code,Contributing_Country,Mission_Acronym,Personnel_Type,Female_Personnel,Male_Personnel,Last_Reporting_Date
431832,ALB,8,Albania,UNMISS,Staff Officer,2,0,30-Nov-20
431833,DZA,12,Algeria,MONUSCO,Experts on Mission,0,2,30-Nov-20
431834,ARG,32,Argentina,MINURSO,Experts on Mission,0,5,30-Nov-20
431835,ARG,32,Argentina,UNFICYP,Troops,19,226,30-Nov-20
431836,ARG,32,Argentina,UNFICYP,Staff Officer,2,6,30-Nov-20
431837,ARG,32,Argentina,UNMISS,Individual Police,1,5,30-Nov-20


In [4]:
contrib$Total_Personnel <- contrib$Female_Personnel + contrib$Male_Personnel
#contrib <- rename (contrib, NAME = Contributing_Country)
tail(contrib)

,Contribution_ID,ISOCode3,M49_Code,Contributing_Country,Mission_Acronym,Personnel_Type,Female_Personnel,Male_Personnel,Last_Reporting_Date,Total_Personnel
151547,411603,NOR,578,Norway,UNMIBH,Formed Police Units,0,12,31-Jul-02,12
151548,411607,USA,840,United States of America (the),UNMIBH,Formed Police Units,0,45,31-Jul-02,45
151549,411611,BEL,56,Belgium,UNMIK,Formed Police Units,0,2,31-Jul-02,2
151550,411615,FRA,250,France,UNMIK,Formed Police Units,0,86,31-Jul-02,86
151551,411619,GRC,300,Greece,UNMIK,Formed Police Units,0,20,31-Jul-02,20
151552,411623,ISL,352,Iceland,UNMIK,Formed Police Units,0,3,31-Jul-02,3


In [5]:
str(contrib)

'data.frame':	151552 obs. of  10 variables:
 $ ���Contribution_ID: int  431832 431833 431834 431835 431836 431837 431838 431839 431840 431841 ...
 $ ISOCode3                   : Factor w/ 148 levels "#N/A","ALB","ARG",..: 2 41 3 3 3 3 3 3 4 4 ...
 $ M49_Code                   : int  8 12 32 32 32 32 32 32 51 51 ...
 $ Contributing_Country       : Factor w/ 147 levels "","Albania","Algeria",..: 2 3 4 4 4 4 4 4 5 5 ...
 $ Mission_Acronym            : Factor w/ 65 levels "BINUB","BINUH",..: 49 15 7 26 26 49 62 64 10 28 ...
 $ Personnel_Type             : Factor w/ 5 levels "Experts on Mission",..: 4 1 1 5 4 3 1 3 4 5 ...
 $ Female_Personnel           : int  2 0 0 19 2 1 0 4 0 2 ...
 $ Male_Personnel             : int  0 2 5 226 6 5 4 8 1 30 ...
 $ Last_Reporting_Date        : Factor w/ 204 levels "28-Feb-03","28-Feb-06",..: 67 67 67 67 67 67 67 67 67 67 ...
 $ Total_Personnel            : int  2 2 5 245 8 6 4 12 1 32 ...


In [6]:
summary(contrib)

 \357\273\277Contribution_ID    ISOCode3         M49_Code    
 Min.   :     1              NPL    :  4560   Min.   :  8.0  
 1st Qu.: 37891              BGD    :  4135   1st Qu.:242.0  
 Median : 75822              GHA    :  3960   Median :458.0  
 Mean   :171730              JOR    :  3808   Mean   :459.4  
 3rd Qu.:331840              NGA    :  3510   3rd Qu.:686.0  
 Max.   :432806              EGY    :  3080   Max.   :894.0  
                             (Other):128499   NA's   :5      
 Contributing_Country Mission_Acronym             Personnel_Type 
 Nepal     :  4560    UNMISS :14709   Experts on Mission :41912  
 Bangladesh:  4135    UNAMID :13671   Formed Police Units:22797  
 Ghana     :  3960    MONUSCO:13012   Individual Police  :32556  
 Jordan    :  3808    MINUSMA: 9035   Staff Officer      :12280  
 Nigeria   :  3510    UNMIL  : 8791   Troops             :42007  
 Egypt     :  3080    MINUSCA: 8154                              
 (Other)   :128499    (Other):84180       

In [7]:
sub <- subset(contrib, Last_Reporting_Date=="30-Nov-20" & Personnel_Type=="Troops")
tail(sub)

,Contribution_ID,ISOCode3,M49_Code,Contributing_Country,Mission_Acronym,Personnel_Type,Female_Personnel,Male_Personnel,Last_Reporting_Date,Total_Personnel
920,432751,TZA,834,Tanzania (United Republic of),UNAMID,Troops,24,300,30-Nov-20,324
924,432755,TZA,834,Tanzania (United Republic of),UNIFIL,Troops,19,97,30-Nov-20,116
940,432771,URY,858,Uruguay,MONUSCO,Troops,69,839,30-Nov-20,908
943,432774,URY,858,Uruguay,UNDOF,Troops,7,203,30-Nov-20,210
951,432782,VNM,704,Viet Nam,UNMISS,Troops,10,53,30-Nov-20,63
954,432785,ZMB,894,Zambia,MINUSCA,Troops,113,797,30-Nov-20,910


In [8]:
chorodata <- group_by(sub, Contributing_Country) %>%
    summarize(Total_Contrib = sum(Total_Personnel))
head(chorodata)

Contributing_Country,Total_Contrib
Argentina,245
Armenia,32
Austria,259
Bangladesh,5924
Belarus,6
Belgium,50


In [ ]:
# Download map data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/
# Download the shapefile. (note that I store it in a folder called DATA. You have to change that if needed.)
#download.file("https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/110m/cultural/ne_110m_admin_0_countries.zip" , destfile="DATA/world_shape_file.zip")

# Unzip this file. 
#system("unzip DATA/world_shape_file.zip")

In [ ]:
# Read this shape file with the rgdal library. 
#library(rgdal
#world_spdf <- readOGR(".", "ne_110m_admin_0_countries")
#summary(world_spdf)

In [ ]:
worldmap = map("world", fill = TRUE, plot = FALSE)
cnames <- data.frame(worldmap$names)
head(cnames)

In [10]:
# Work with the dates
library(lubridate)
as_date("30-Nov-20")


Attaching package: 'lubridate'

The following object is masked from 'package:base':

    date



[1] "2030-11-20"

In [11]:
contrib$MonthYear <- parse_date_time(contrib$Last_Reporting_Date, "dmy") 
contrib$MonthYear <-  floor_date(contrib$MonthYear, unit="month")

In [12]:
contrib

Contribution_ID,ISOCode3,M49_Code,Contributing_Country,Mission_Acronym,Personnel_Type,Female_Personnel,Male_Personnel,Last_Reporting_Date,Total_Personnel,MonthYear
431832,ALB,8,Albania,UNMISS,Staff Officer,2,0,30-Nov-20,2,2020-11-01
431833,DZA,12,Algeria,MONUSCO,Experts on Mission,0,2,30-Nov-20,2,2020-11-01
431834,ARG,32,Argentina,MINURSO,Experts on Mission,0,5,30-Nov-20,5,2020-11-01
431835,ARG,32,Argentina,UNFICYP,Troops,19,226,30-Nov-20,245,2020-11-01
431836,ARG,32,Argentina,UNFICYP,Staff Officer,2,6,30-Nov-20,8,2020-11-01
431837,ARG,32,Argentina,UNMISS,Individual Police,1,5,30-Nov-20,6,2020-11-01
431838,ARG,32,Argentina,UNTSO,Experts on Mission,0,4,30-Nov-20,4,2020-11-01
431839,ARG,32,Argentina,UNVMC,Individual Police,4,8,30-Nov-20,12,2020-11-01
431840,ARM,51,Armenia,MINUSMA,Staff Officer,0,1,30-Nov-20,1,2020-11-01
431841,ARM,51,Armenia,UNIFIL,Troops,2,30,30-Nov-20,32,2020-11-01


In [14]:
max(contrib$MonthYear)
min(contrib$MonthYear)
summary(contrib$MonthYear)

[1] "2020-11-01 UTC"

[1] "2002-07-01 UTC"

                 Min.               1st Qu.                Median 
"2002-07-01 00:00:00" "2010-07-01 00:00:00" "2014-04-01 00:00:00" 
                 Mean               3rd Qu.                  Max. 
"2013-10-06 09:57:51" "2017-10-01 00:00:00" "2020-11-01 00:00:00" 

In [15]:
graph_data <- subset(contrib, Contributing_Country=="Argentina" & MonthYear==as_date("2020-11-01"))%>%
group_by(Personnel_Type)%>%summarize(Total = sum(Total_Personnel))
graph_data

Personnel_Type,Total
Experts on Mission,9
Individual Police,18
Staff Officer,8
Troops,245


In [ ]:
lon <- 63.6
lat <- -38.4

      # find the country name using the maps package
cn <- map.where("world", lon, lat)
cn

In [ ]:
map.where("world", x = -31.41, y = -64.18)

In [ ]:
subset(contrib, contrib$MonthYear == as_date("2014-04-01"))